# Normalização dos Corpora

Este notebook cria o modelo normalizado dos corpora compilados, de forma que:
- Constrói o modelo a partir das sentenças do córpus
- Faz o uso do modelo obtido para consultar similaridade de palavras
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [1]:
import os
os.chdir('..')

-------------------

## Modelo Word2Vec

Carregar a biblioteca do modelo ```Word2Vec``` e do normalizador:

In [2]:
from gensim.models import Word2Vec
from nlputils.lexical.normalizer import Normalizer

Instanciar um objeto para fazer as normalizações necessárias:

In [3]:
normalizer = Normalizer()

Implementar uma função para auxiliar no carregamento das sentenças a serem usadas no treinamento do modelo. 

A função ```load_sentences``` recebe como entrada a string indicando o path do diretório do córpus a ser usado, retornando a lista contendo todas as sentenças desse córpus:

In [4]:
def load_sentences(corpus_dir):   
    all_sentences = []
    for file_path in os.listdir(corpus_dir):
        with open(corpus_dir + file_path) as fp:
            for line in fp:
                line = normalizer.to_lowercase(line)
                sentences = normalizer.tokenize_sentences(line)
                sentences = [normalizer.tokenize_words(sent) for sent in sentences]
                all_sentences.extend(sentences)
    return all_sentences

----------------

## Word2Vec do Córpus Telefonia

Primeiro, carregar todas as sentenças do córpus:

In [5]:
telefonia_sentences = load_sentences('data/corpora/telefonia/')

Depois, usar as sentenças obtidas como treinamento do modelo, que será instanciado em ```telefonia_w2vmodel```:

In [6]:
telefonia_w2vmodel = Word2Vec(telefonia_sentences, size=200, window=5, min_count=3, workers=4)

### Usando o modelo construído

Com o modelo construído, pode ser feita a consulta das 10 palavras mais similares de uma dada palavras, usando o método ```wv.most_similar```. A seguir e dado o exemplo de uso com três palavras:

In [7]:
words = ['telefonia', 'sistema', 'internet']

for word in words:
    top10 = telefonia_w2vmodel.wv.most_similar(word)
    print(word, '==>', ', '.join([x[0] for x in top10]))

telefonia ==> base, patentes, américa, tim, apenas, mil, segurança, graças, rede, espaço
sistema ==> fora, pelas, presidente, processo, xperia, vender, depois, todo, usar, toda
internet ==> após, segurança, espaço, serviços, problemas, protótipos, foram, display, agência, tim


### Comentários acerca dos resultados do modelo

- O modelo é impreciso, devido ao córpus ser composto por poucos documentos.
- Além disso, a formatação dos documentos não é perfeita, existem palavras que estão concatenadas entre si ou com sinais de ponto final, o que deixa muita sujeira nos dados do treinamento.

---------------

## Word2Vec do Córpus Saúde

Primeiro, carregar todas as sentenças do córpus:

In [8]:
saude_sentences = load_sentences('data/corpora/saude/')

Depois, usar as sentenças obtidas como treinamento do modelo, que será instanciado em ```telefonia_w2vmodel```:

In [9]:
saude_w2vmodel = Word2Vec(saude_sentences, size=200, window=5, min_count=3, workers=4)

### Usando o modelo construído

Com o modelo construído, pode ser feita a consulta das 10 palavras mais similares de uma dada palavras, usando o método ```wv.most_similar```. A seguir e dado o exemplo de uso com três palavras:

In [10]:
words = ['saúde', 'doença', 'prevenção']

for word in words:
    top10 = saude_w2vmodel.wv.most_similar(word)
    print(word, '==>', ', '.join([x[0] for x in top10]))

saúde ==> sigla, mundial, da, opas, aquiraz, brasileira, universidade, organização, sociedade, sanitária
doença ==> criança, paciente, previsão, pessoa, cicatrização, ver, jeito, certamente, desafio, esse
prevenção ==> criação, grupo, cfm, posicionamento, infantil, chegou, entrada, maconha, plasma, pressão


### Comentários acerca dos resultados do modelo

- O modelo é impreciso, devido ao córpus ser composto por poucos documentos.
- Além disso, a formatação dos documentos não é perfeita, existem palavras que estão concatenadas entre si ou com sinais de ponto final, o que deixa muita sujeira nos dados do treinamento.

-----------------

## Modelo Doc2Vec

Carregar as bibliotecas do modelo ```Doc2Vec``` e ```scipy``` para cálculos:

In [11]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy import spatial

Implementar função para facilitar obtenção de todos os documentos dos corpora:

In [12]:
def corpus_documents(corpus_dir):
    all_documents = []
    for file in os.listdir(corpus_dir):
        with open(corpus_dir + file) as text_file:
            document = ' '.join(text_file.readlines())
            document = normalizer.to_lowercase(document)
            # document = normalizer.remove_ponctuations(document)
            document_tokens = normalizer.tokenize_words(document)
            # document_tokens = normalizer.remove_stopwords(document_tokens)
            all_documents.append(document_tokens)
    return all_documents

Juntar os documentos dos dois córpus, telefonia e saúde, para montar a tag que será usada na construção do modelo Doc2Vec:

In [13]:
all_documents = []
all_documents.extend(corpus_documents('data/corpora/telefonia/'))
all_documents.extend(corpus_documents('data/corpora/saude/'))

tagged_documents = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(all_documents)]

Construção do modelo, atribuído em ```d2vmodel```:

In [14]:
d2vmodel = Doc2Vec(tagged_documents, vector_size=20, window=2, min_count=1, workers=4)

### Usando o modelo construído

O modelo criado pode ser usado para comparar o grau de semelhança entre dois documentos. Para isso, será usada a função ```similarity```, que recebe dois vetores inferidos pelo modelo e retorna o valor calculado de semelhança (entre 0 e 1):

In [15]:
def similarity(vector1, vector2):
    return 1 - spatial.distance.cosine(vector1, vector2)

Será exemplificado o uso usando três documentos:

In [16]:
doc1 = all_documents[10]
doc2 = all_documents[1]
doc3 = all_documents[-1]

vector_1 = d2vmodel.infer_vector(doc1)
vector_2 = d2vmodel.infer_vector(doc2)
vector_3 = d2vmodel.infer_vector(doc3)

Conteúdo do ```doc1```:

In [17]:
print(' '.join(doc1))

como sabemos , a filha do fundador da huawei está presa no país norte-americano desde dezembro . assim , os eua pedem que ela responda em seu território a todas as acusações como espionagem , fraude bancária e violação de sanções contra o irã.além disso , o processo também envolve duas empresas que são subsidiárias da huawei e que , segundo o governo dos eua , foram usadas para promover a espionagem industrial no país e furar o bloqueio comercial imposto ao país árabe.segundo promotores do caso , a huawei ainda roubou informações sobre um robô que a operadora t-mobile usa para testar smartphones . eles citam que engenheiros da empresa fotografaram , mediram e até pegaram um componente usado pelo dispositivo.além disso , a fabricante chinesa também é acusada de premiar funcionários que roubam informações de empresas concorrentes . de acordo com christopher wray , diretor de investigações , o processo é um avanço contra a conduta da huawei : o caso expõem as ações descaradas e persistent

Conteúdo do ```doc2```:

In [18]:
print(' '.join(doc2))

assim como estamos noticiando há algum tempo , ao que tudo indica a apple deverá adotar de fato um design um tanto quanto curioso – para não dizer esquisito – com relação a parte traseira dos novos iphones xi e xi max , aguardados para o terceiro trimestre desse ano , trazendo 3 câmeras traseiras arranjadas em um layout triangular.agora , após renderizações não oficiais liberadas ainda ontem terem revelado o design do modelo branco com sua câmera tripla e seu design frontal reciclado , novas imagens e um vídeo nos mostram de perto todos os detalhes desse arranjo fotográfico singular ; confira o vídeo logo abaixo.como podemos observar no vídeo – e em melhores detalhes nas imagens a seguir , as câmeras dos novos iphones ficarão alocadas em uma espécie de lombada mais suave , com curvas feitas em vidro que saem da superfície plana do resto do painel traseiro para o arranjo de câmeras , em um design que lembra um pouco o que víamos no iphone 7 e 7 plus , sendo a única diferença o material 

Conteúdo do ```doc3```:

In [19]:
print(' '.join(doc3))

não existe nível seguro de consumo de álcool , mostra pesquisapesquisa diz que beber moderadamente pode proteger o coração , mas risco de câncer e outros males se sobrepõecopiar link24.ago.2018 às 18h04ouvir o textolaurel ivesuma má notícia para quem gosta de tomar uma taça de vinho no fim do dia , acreditando ser um hábito saudável.um novo estudo global , publicado na revista científica the lancet , confirmou o que algumas pesquisas anteriores diziam : não existe um nível seguro para o consumo de álcool.os pesquisadores admitem que beber moderadamente pode proteger contra doenças cardíacas , mas sugerem que o risco de desenvolver câncer e outros males se sobrepõe aos benefícios.de acordo com os autores do estudo , essas descobertas são as mais significativas já realizadas até hoje , devido à variedade de fatores levados em conta na pesquisa.​estudo indica que tomar uma dose de bebida por dia também aumenta os riscos para a saúde - getty imagesquão arriscado é beber moderadamente ? o e

Cálculo de similariade entre os textos:

In [20]:
print(similarity(vector_1, vector_2))

0.642591118812561


In [21]:
print(similarity(vector_1, vector_3))

0.9661234021186829


In [22]:
print(similarity(vector_2, vector_3))

0.6401811242103577


### Comentários acerca dos resultados do modelo

- Os resultados obtidos nas comparações do Doc2Vec não foram satisfatórios, uma vez que mostrou alto grau de similaridade entre entre ```doc1``` e ```doc3```, o que não faz sentido, pois o primeiro fala sobre polêmicas da Hawuei e o segundo sobre consumo de álcool.
- Foi feito o mesmo procedimento, mas fazendo normalização removendo pontuações e stopwords, mas os valores de similaridade continuaram bem parecidos com o anterior.